In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
#Generates indoor and outdoor temperature statistics
# Define paths

path_LT_Pulses_Flag = ... #Origin folder of LT HP Pulses Flag as generated from generate_power_data_for_LT_ASHP.ipynb
path_LT_Temperature_Statistics = ... #Destination folder for LT Temperature Statistics

# Load dataset the path where the modified all_summary.csv file is located 
all_summary = pd.read_csv('all_summary.csv') 

# Filter data
all_summary = all_summary[all_summary['Included_SPF_analysis'] == True]
all_summary_LT_ASHP = all_summary[all_summary['HP_Installed'] == "ASHP"]

# Process each property
for i, property_id in enumerate(all_summary_LT_ASHP['Property_ID']):
    file_path = os.path.join(path_LT_Pulses_Flag, f"{property_id}.parquet")
    
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue

    # Load household data
    household = pd.read_parquet(file_path)

    # Convert timestamp to datetime
    household['Timestamp'] = pd.to_datetime(household['Timestamp'])

    # Extract time-based attributes
    household['Year'] = household['Timestamp'].dt.year
    household['Month'] = household['Timestamp'].dt.month
    household['Day'] = household['Timestamp'].dt.day
    household['DayOfYear'] = household['Timestamp'].dt.dayofyear

    # Initialize statistics dataframe
    temperature_statistics = []

    # Process yearly and daily data
    for year_iter in range(household['Year'].min(), household['Year'].max() + 1):
        household_year = household[household['Year'] == year_iter]

        for dayofyear_iter in range(household_year['DayOfYear'].min(), household_year['DayOfYear'].max() + 1):
            household_day = household_year[household_year['DayOfYear'] == dayofyear_iter]

            if household_day.empty:
                continue  # Skip empty days

            # Compute daily statistics
            avg_temp_ext = household_day['External_Air_Temperature'].mean()
            min_temp_ext = household_day['External_Air_Temperature'].min()
            max_temp_ext = household_day['External_Air_Temperature'].max()
            avg_temp_int = household_day['Internal_Air_Temperature'].mean()
            min_temp_int = household_day['Internal_Air_Temperature'].min()
            max_temp_int = household_day['Internal_Air_Temperature'].max()

            timestamp = datetime(year_iter, 1, 1) + timedelta(days=dayofyear_iter - 1)

            temperature_statistics.append([
                timestamp, avg_temp_ext, min_temp_ext, max_temp_ext,
                avg_temp_int, min_temp_int, max_temp_int
            ])

    # Convert to DataFrame
    columns = ['Time', 'avg_temp_ext', 'min_temp_ext', 'max_temp_ext', 'avg_temp_int', 'min_temp_int', 'max_temp_int']
    temperature_statistics_df = pd.DataFrame(temperature_statistics, columns=columns)

    # Save to Parquet
    output_path = os.path.join(path_LT_Temperature_Statistics, f"{property_id}.parquet")
    temperature_statistics_df.to_parquet(output_path, index=False)

    print(f"Processed and saved: {output_path}")